---------- Start of generic setup code to get kaggle credentials login and data to work with. Copied much from Workshop 2 ----------

In [1]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

https://drive.google.com/file/d/1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x

In [3]:
#GET data from file I've uploaded to my drive. Use your own kaggle.json and google auth logins etc
downloaded = drive.CreateFile({'id':'1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x'}) 
downloaded.GetContentFile('predict-energy-consumption.zip')

In [4]:
!unzip predict-energy-consumption.zip

!ls

Archive:  predict-energy-consumption.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
predict-energy-consumption.zip	sample_submission.csv  train.csv
sample_data			test.csv


In [5]:
from google.colab import files

files.upload(); # upload your kaggle.json file

Saving kaggle.json to kaggle.json


Processing data from the source.

In [6]:
import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [7]:
#GENERIC SUBMIT FUNCTION (uncomment when you want to submit)

#submission = pd.DataFrame(
#    {'key': test.key, 'fare_amount': val_predictions},
#    columns = ['key', 'fare_amount'])
#submission.to_csv('submission.csv', index = False)


#!kaggle competitions submit -c predict-electricity-consumption -f submission.csv -m "[TEAM NAME] Submission"

In [8]:
#------------------ ACTUAL PROBLEM SOLVING BELOW ---------------------------

import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
 
#Here we can split into train/test for verification while we train whatever works)
train =  pd.read_csv('train.csv')
train_labels = train['consumption'] #y vals in this case are concumption

#GIVEN time is a string we just gonna drop that mfer, cbf processing it etc
train = train.drop(['time'], 1)
train = train.drop(['consumption'], 1)
#print(train)

test = pd.read_csv('test.csv')
test = test.drop(['time'],1)
#Have to predict consumption for these values. Consumption unkown.

train_labels = train_labels.replace(np.nan, 0.0)

for col in train.columns:
  train[col] = train[col].replace(np.nan, 0.0)

for col in test.columns:
  test[col] = test[col].replace(np.nan, 0.0)

X_train, X_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.2, random_state=8080)
#print(train)
#print(test)

#At this point should have a train and test without Nan values:)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


---------- End of generic setup code. Actual programming can begin after here ----------

In [9]:
#PCA to get a lesser number of features, then going to feed into basic nn see what we can do
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#0.9 = 2, 0.95 = 3, 0.96 = 4. Through trial and error. 0.95 seems appropriate.
pca = PCA(n_components = 0.95) #number adjusted manually to get results ^

pca.fit(X_train)

frankenTrained = pca.transform(X_train)
frankenTest = pca.transform(X_test)

#print(frankenTrained)

#plt.plot(frankenTrained)

In [10]:
#some basic classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#todo, can play with these values
for r in range(1,10):
  rf = RandomForestRegressor(max_depth=r, random_state = 8080)
  #frankedTrained contains nans or some shit
  rf.fit(frankenTrained, y_train)

  #output = rf.predict(frankenTest)
  output = rf.predict(frankenTest)
  #print(output)
  #(y_test)

  #performance ~0.05 worse for using only 3 features vs 11, meaning three seems pretty decent
  res = mean_squared_error(output,y_test,squared = False)
  print(res) #we see that using a base classifier on 3 features a mse of ~1.1 is standard

1.1203894002342503
1.1167556503101073
1.1113389794367865
1.1070042514836562
1.1024576743869947
1.1020577678466226
1.098942521830455
1.0956014397519125
1.0921605535911996


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#reference material: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
def basicModel():
  model = Sequential()
  model.add(Dense(3, input_shape=(3,), kernel_initializer='normal', activation='relu'))
  model.add(Dense(10, kernel_initializer='normal', activation='relu'))
  model.add(Dense(10, kernel_initializer='normal', activation='relu'))
  model.add(Dense(10, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=basicModel(), epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=6)  
results = cross_val_score(pipeline, frankenTrained, y_train, cv=kfold, scoring='neg_mean_squared_error')

#NOTE: go grab a coffee or have a smoke - this takes about 13 minutes to run.
print(results) #we see a generic NN performs no better than RF on the same 3 features

[-0.9935131  -1.1394648  -1.36147595 -1.11644757 -1.18449053 -1.13226669]
